## 4.1 Ord antal og histogrammer


### 4.1.1 Angiv udvælgelseskriterier

In [ ]:
!!pip install pytrends

['Collecting pytrends',
 '  Downloading https://files.pythonhosted.org/packages/96/53/a4a74c33bfdbe1740183e00769377352072e64182913562daf9f5e4f1938/pytrends-4.7.3-py3-none-any.whl',
 'Requirement already satisfied: lxml in /usr/local/lib/python3.6/dist-packages (from pytrends) (4.2.6)',
 'Requirement already satisfied: pandas>=0.25 in /usr/local/lib/python3.6/dist-packages (from pytrends) (1.1.5)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.6/dist-packages (from pytrends) (2.23.0)',
 'Requirement already satisfied: pytz>=2017.2 in /usr/local/lib/python3.6/dist-packages (from pandas>=0.25->pytrends) (2018.9)',
 'Requirement already satisfied: python-dateutil>=2.7.3 in /usr/local/lib/python3.6/dist-packages (from pandas>=0.25->pytrends) (2.8.1)',
 'Requirement already satisfied: numpy>=1.15.4 in /usr/local/lib/python3.6/dist-packages (from pandas>=0.25->pytrends) (1.19.5)',
 'Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/

### 4.1.2 Vælg tabeller / filer af interesse

### 4.1.3 Vælg data i en tabel / fil


In [ ]:
import pandas as pd
import numpy as np
from pytrends.request import TrendReq
pytrends = TrendReq(hl="en-US", tz=360)
recession_list=['Agurk','Paprika','Brokkoli','Blomkål','Hodekål','Kålrot','Kinakål','Rosenkål','Isbergsalat','Purre','Sjampinjong','Banan','Drue','Melon','Kiwi','Ananas','Pære','Appelsin','Jordbær','Blåbær','Bringebær','Fersken']

def gtrends_overtime(full_list, key_ref, save_name="", directory="", category=0, time='all', loc=''):
    i = 0
    database=pd.DataFrame()
    while i < len(recession_list):
        l = recession_list[i:(i+4)]
        l.append(key_ref)
        l=[w.replace('_', ' ') for w in l]
        pytrends.build_payload(l, cat=category, timeframe=time, geo=loc, gprop='')
        df_time = pytrends.interest_over_time()
        df_time.drop(df_time.columns[len(df_time.columns)-1],axis=1, inplace=True)
        database=pd.concat([database,df_time],join="outer", axis=1)      
        i += 4
    return database
data=gtrends_overtime(recession_list, "Pizza", "_worldwide_",directory="",  category=71, time='today 5-y', loc="NO")
data['date']=data.index
data

,Agurk,Paprika,Brokkoli,Blomkål,Pizza,Hodekål,Kålrot,Kinakål,Rosenkål,Pizza,Isbergsalat,Purre,Sjampinjong,Banan,Pizza,Drue,Melon,Kiwi,Ananas,Pizza,Pære,Appelsin,Jordbær,Blåbær,Pizza,Bringebær,Fersken,Pizza,date
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-02-21,2,18,8,5,56,0,0,0,2,56,0,3,2,6,56,1,1,20,2,46,1,7,3,8,56,7,0,56,2016-02-21
2016-02-28,1,17,10,8,63,1,3,0,1,63,0,0,1,8,63,1,1,19,2,51,0,8,8,1,63,4,2,63,2016-02-28
2016-03-06,4,6,11,8,64,0,3,0,2,64,0,2,1,10,64,0,1,18,2,53,2,13,6,3,64,5,1,64,2016-03-06
2016-03-13,2,14,6,8,41,2,1,0,1,41,1,4,2,9,41,1,2,24,0,34,2,19,5,2,41,8,1,41,2016-03-13
2016-03-20,2,5,7,5,71,0,1,1,4,71,0,2,0,7,71,0,4,43,2,58,3,26,7,2,71,7,0,71,2016-03-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-17,3,10,14,14,87,1,3,0,3,87,0,1,2,19,87,1,2,29,1,71,2,8,5,3,87,3,0,87,2021-01-17
2021-01-24,4,12,13,11,84,1,1,1,2,84,0,3,1,16,84,1,1,32,2,68,0,11,6,10,84,6,1,84,2021-01-24
2021-01-31,4,5,8,11,84,1,3,0,2,84,0,1,1,9,84,1,1,30,2,69,3,9,6,6,84,6,1,84,2021-01-31


In [ ]:

data.replace([np.inf, -np.inf], np.nan)
data=data.dropna()

data = data.loc[:,~data.columns.duplicated()]


labels=list(data.columns.unique().drop(['date']))

import altair as alt

brush_sel = alt.selection(type='interval', encodings=['x'])

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['x'], empty='none')

lower=alt.Chart(data).transform_fold(labels).mark_line().encode(
    y=alt.Y('value:Q'),
    x='date:T',
    color='key:N'
).properties(width=800,height=300).add_selection(brush_sel)

upper=lower.transform_window(
    rolling_mean='mean(value)',
    frame=[-2, 2]
).mark_bar(opacity=0.9).encode(
    y=alt.Y("value:Q"), #,sort='-y' , stack=None
    x='yearmonth(date):O',
    color='key:N',
    
    tooltip=alt.Tooltip(['key:N','value:N']),
).properties(width=800,height=300).transform_filter(brush_sel)

selectors = alt.Chart(data).transform_fold(labels).mark_point().encode(
    x='value:Q',
    color='key:N',
    opacity=alt.value(0),
).add_selection(
    nearest
)

up=alt.layer(upper)


alt.vconcat(up,lower)

alt.VConcatChart(...)